# Vectorización de texto
### Creación de los dataframe y limpieza de texto

In [1]:
import pandas as pd
import numpy as np
from pprint import pprint

df = pd.read_csv('titulosEncontrados.csv', sep=',',  encoding='utf-8')
#convierte en string los datos de la columna text
df1 = df['titulo'].apply(str)

# Fase de limpieza

In [2]:
#Limpieza de caracteres no deseados
import nltk
import re

def general(txt: str, bert=False, nums=False) -> str:
    """
    Elimina caracteres no deseados
    Params:
        **txt**:texto a ser limpiado de caracteres no desaeados
    """
    if nums:
        txt = re.sub(r'\d+', ' ', txt)
    if not bert:
        txt = txt.translate(str.maketrans(
            'áéíóúýàèìòùÁÉÍÓÚÀÈÌÒÙÝ', 'aeiouyaeiouAEIOUAEIOUY'))
        txt = re.sub(r'[^\w\s:.)-]', '', txt)

    txt = txt.replace('\r', ' ').replace("\v", ' ').replace(
        "\t", ' ').replace("\f", ' ').replace("\a", ' ').replace("\b", ' ')
    txt = re.sub(' +', ' ', txt)
    txt = txt.strip()
    return txt

dfLimpio = df1.apply(general)


In [47]:
#limpieza de stopwords

stops = nltk.corpus.stopwords.words('spanish')
# for i in stops:
#     stops.append(i.upper())


def remove_stops(texto: str) -> str:
    """
    Función que elimina stopwords
    Params:
        **texto**:texto a ser limpiado de stopwords

    """
    texto = [
        i for i in texto.split() if i not in stops
    ]
    return ' '.join(texto)

dfLimpio = dfLimpio.apply(remove_stops)
dfLimpio = dfLimpio.apply(str)

# TF-IDF

In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=25,
                                 min_df=100, stop_words=stops,
                                 use_idf=True, ngram_range=(1,5))

tfidf_matrix = tfidf_vectorizer.fit_transform(dfLimpio) #fit the vectorizer to dfLimpio

print(tfidf_matrix.shape)

(5390, 25)


In [50]:
terms = tfidf_vectorizer.get_feature_names_out()

In [51]:
terms

array(['actor', 'antecedentes', 'antecedentes autos', 'autos', 'clinica',
       'conclusiones', 'consideraciones', 'consideraciones medico',
       'consideraciones medico legales', 'contestacion',
       'contestacion puntos', 'examen', 'historia', 'historia clinica',
       'interes', 'interes medico', 'legal', 'legales', 'medico',
       'medico legal', 'medico legales', 'pericia', 'petitorio', 'puntos',
       'puntos pericia'], dtype=object)

In [52]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)

In [53]:
print(dist)

[[ 1.00000000e+00  1.00000000e+00  1.00000000e+00 ...  1.00000000e+00
   1.00000000e+00  1.00000000e+00]
 [ 1.00000000e+00 -2.22044605e-16  1.00000000e+00 ...  1.00000000e+00
   1.00000000e+00  8.75989722e-01]
 [ 1.00000000e+00  1.00000000e+00 -2.22044605e-16 ...  1.00000000e+00
   1.00000000e+00  1.00000000e+00]
 ...
 [ 1.00000000e+00  1.00000000e+00  1.00000000e+00 ...  1.00000000e+00
   1.00000000e+00  1.00000000e+00]
 [ 1.00000000e+00  1.00000000e+00  1.00000000e+00 ...  1.00000000e+00
   0.00000000e+00  1.00000000e+00]
 [ 1.00000000e+00  8.75989722e-01  1.00000000e+00 ...  1.00000000e+00
   1.00000000e+00  0.00000000e+00]]


# K-means clustering

In [9]:
from sklearn.cluster import KMeans

num_clusters = 10

km = KMeans(n_clusters=num_clusters)

km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

# Pruebas de tfidf.


In [11]:
# creamos el bag of words
word2count = {}  
for data in dfLimpio:  
    words = nltk.word_tokenize(data)  
    for word in words:  
        if word not in word2count.keys():  
            word2count[word] = 1
        else:  
            word2count[word] += 1

In [22]:
# calcular la puntuación tf-idf para cualquier corpus
from sklearn.feature_extraction.text import TfidfVectorizer 
import pandas as pd 
  
tfidf = TfidfVectorizer(min_df = 100, max_df = 0.80, ngram_range = (1, 3), max_features=25) 
features = tfidf.fit_transform(dfLimpio) 
  
dfTfIdf = pd.DataFrame(features.todense(), columns = tfidf.get_feature_names_out())

In [25]:
# kMeans
from sklearn.cluster import KMeans

num_clusters = 10

km = KMeans(n_clusters=num_clusters)

km.fit(features)

clusters = km.labels_.tolist()

In [76]:
# pruebas de collocation - n-grams
import nltk

from nltk.collocations import *

bi_gram= nltk.collocations.BigramAssocMeasures()

Collocation = BigramCollocationFinder.from_words(dfLimpio)

Collocation.nbest(bi_gram.pmi, 20)

[('abor atencion publico ocasiones', 'sesiones'),
 ('actitud general', 'puntos'),
 ('actividad laboral', 'existen constancias'),
 ('actor', 'actualidad existe patologia'),
 ('actor antecedentes patologias previas', 'otorgo prestaciones tiempo forma'),
 ('actor debio intervenido', 'actualidad requiere tratamiento cirugia'),
 ('actor necesita', 'efectivamente trabajador puede'),
 ('actor padece sintomatologia relatada',
  'requiere recalificacion profesional'),
 ('actor posee algun', 'punto vista psiquiatrico'),
 ('actor presenta actualidad', 'actor'),
 ('actor presenta algun grado secuela indole',
  'considera necesidad tratamiento'),
 ('actor presenta momento actual patologia', 'acarrea'),
 ('actor recibio adecuada atencion medica', 'actor posee algun'),
 ('actor solicito apoyo psicoterapeutico', 'posee personalidad'),
 ('actora presenta momento actual patologia',
  'patologia guarda relacion accidente'),
 ('actualidad existe patologica psiquiatrica', 'hector javier galeno'),
 ('adjunt

In [83]:
text = dfLimpio.apply(remove_stops)

tokens = nltk.wordpunct_tokenize(str(text))

finder = BigramCollocationFinder.from_words(tokens)

sorted(finder.nbest(bi_gram.raw_freq,10))

[(',', 'Length'),
 (',', 'dtype'),
 ('...', '5385'),
 ('0', 'proemio'),
 ('1', 'antecedentes'),
 ('2', 'examen'),
 ('3', 'consideraciones'),
 ('4', 'conclusiones'),
 ('5385', 'petitorio'),
 ('medico', 'legales')]

In [ ]:
from nltk.corpus import ieer
docs = ieer.parsed_docs('NYT_19980315')
tree = docs[1].text
print(tree)

In [73]:
from nltk.sem import relextract
pairs = relextract.tree2semi_rel(dfLimpio)
for s, tree in pairs[18:22]:
    print('("...%s", %s)' % (" ".join(s[-5:]),tree))

In [75]:
import spacy
nlp = spacy.load("es_core_news_sm")


In [94]:
tokenize = nltk.word_tokenize(str(dfLimpio))

In [95]:
tag  = nltk.pos_tag(tokenize)

In [96]:
tag

[('0', 'CD'),
 ('proemio', 'JJ'),
 ('1', 'CD'),
 ('antecedentes', 'NNS'),
 ('interes', 'NNS'),
 ('medico', 'VBP'),
 ('legal', 'JJ'),
 ('2', 'CD'),
 ('examen', 'NNS'),
 ('fisico', 'JJ'),
 ('actor', 'NN'),
 ('3', 'CD'),
 ('consideraciones', 'NNS'),
 ('medico', 'VBD'),
 ('legales', 'NNS'),
 ('4', 'CD'),
 ('conclusiones', 'NNS'),
 ('...', ':'),
 ('5385', 'CD'),
 ('petitorio', 'NN'),
 ('5386', 'CD'),
 ('historia', 'NN'),
 ('clinica', 'NN'),
 ('5387', 'CD'),
 ('estudios', 'NNS'),
 ('solicitados', 'JJ'),
 ('5388', 'CD'),
 ('puntos', 'NN'),
 ('periciales', 'NNS'),
 ('5389', 'CD'),
 ('consideraciones', 'NNS'),
 ('conclusiones', 'NNS'),
 ('medico', 'VBP'),
 ('legales', 'NNS'),
 ('Name', 'NN'),
 (':', ':'),
 ('titulo', 'NN'),
 (',', ','),
 ('Length', 'NNP'),
 (':', ':'),
 ('5390', 'CD'),
 (',', ','),
 ('dtype', 'NN'),
 (':', ':'),
 ('object', 'NN')]